# Part 03 - Remote Execution Exercise  - Solution

In the last notebook, we trained a model to classify images of clothing. Now let's remotely train a model to perform sentiment analysis on the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb).

In [1]:
import tensorflow as tf
import syft as sy

hook = sy.TensorFlowHook(tf)
bob = sy.VirtualWorker(hook, id="bob")

The dataset has already been processed, so now you just have to send the data to the virtual worker 'Bob' and train your model.

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

x_train, y_train = tf.convert_to_tensor(x_train), tf.convert_to_tensor(y_train)
x_test, y_test = tf.convert_to_tensor(x_test), tf.convert_to_tensor(y_test)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


## Train your model with remote execution:

In [3]:
# Send the data to
x_train_ptr = x_train.send(bob)
y_train_ptr = y_train.send(bob)

In [4]:
# Source code https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
model_ptr = model.send(bob)

model_ptr.fit(x_train_ptr, y_train_ptr, 
              batch_size=batch_size, 
              epochs=15, 
              validation_split=0.1)

W1023 21:41:51.983034 4316575168 deprecation.py:506] From /anaconda3/envs/tf20/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 22500 samples, validate on 2500 samples
Epoch 1/15
22500/22500 [==============================] - 59s 3ms/sample - loss: 0.4345 - accuracy: 0.7987 - val_loss: 0.3975 - val_accuracy: 0.8232
Epoch 2/15
22500/22500 [==============================] - 75s 3ms/sample - loss: 0.2530 - accuracy: 0.8978 - val_loss: 0.3799 - val_accuracy: 0.8244
Epoch 3/15
22500/22500 [==============================] - 74s 3ms/sample - loss: 0.1665 - accuracy: 0.9387 - val_loss: 0.4993 - val_accuracy: 0.8184
Epoch 4/15
22500/22500 [==============================] - 61s 3ms/sample - loss: 0.1076 - accuracy: 0.9610 - val_loss: 0.5065 - val_accuracy: 0.8208
Epoch 5/15
22500/22500 [==============================] - 60s 3ms/sample - loss: 0.0811 - accuracy: 0.9710 - val_loss: 0.6291 - val_accuracy: 0.8348
Epoch 6/15
22500/22500 [==============================] - 72s 3ms/sample - loss: 0.0599 - accuracy: 0.9804 - val_loss: 0.7197 - val_accuracy: 0.8188
Epoch 7/15
22500/22500 [==============================] -